<a href="https://colab.research.google.com/github/econ105/AI/blob/main/Python/Algotrading/telegram/performance_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install python-telegram-bot --upgrade

In [5]:
import asyncio
from telegram import Bot

TELEGRAM_BOT_TOKEN = "1666447235:AAHGohNJqq-PbDlKlG-SJTigQbgU7D8w9jU"
TELEGRAM_CHAT_ID = "612618566"   # your chat id

async def main():
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    await bot.send_message(chat_id=TELEGRAM_CHAT_ID, text="hi this is testing")

# Jupyter-safe way to run the coroutine
loop = asyncio.get_event_loop()
if loop.is_running():
    # For Jupyter/IPython: schedule and wait
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())

In [6]:
import asyncio
from datetime import datetime, timedelta

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from telegram import Bot

TELEGRAM_BOT_TOKEN = "1666447235:AAHGohNJqq-PbDlKlG-SJTigQbgU7D8w9jU"
TELEGRAM_CHAT_ID = "612618566"  # use integer, your chat id

TICKERS = ["HL", "SLV"]
PERIOD_DAYS = 60  # ~ last 2 months
CHART_FILENAME = "rel_perf_HL_SLV.png"


def download_data(tickers, period_days=60):
    end = datetime.today()
    start = end - timedelta(days=period_days)
    data = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        interval="1d",
        auto_adjust=False,
        progress=False,
    )
    # Handle single vs multi-index columns
    if isinstance(data.columns, pd.MultiIndex):
        close = data["Close"].copy()
    else:
        close = data[["Close"]].copy()
    close = close.dropna(how="all")
    return close


def compute_relative_performance(close_df):
    rel = close_df.divide(close_df.iloc[0]) * 100
    return rel


def make_relative_chart(rel_df, chart_path):
    plt.style.use("seaborn-v0_8-whitegrid")

    fig, ax = plt.subplots(figsize=(10, 6))

    for col in rel_df.columns:
        ax.plot(rel_df.index, rel_df[col], label=col)

    ax.set_title("Relative Performance (Last ~2 Months, Close = 100 at Start)")
    ax.set_xlabel("Date")
    ax.set_ylabel("Relative Performance (Start = 100)")
    ax.legend(title="Ticker", loc="best")

    fig.autofmt_xdate()
    fig.tight_layout()
    fig.savefig(chart_path, dpi=150)
    plt.close(fig)


async def main():
    # 1) test text message
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    await bot.send_message(chat_id=TELEGRAM_CHAT_ID, text="hi this is testing")

    # 2) data and chart
    close = download_data(TICKERS, PERIOD_DAYS)
    rel = compute_relative_performance(close)
    make_relative_chart(rel, CHART_FILENAME)

    # 3) send chart
    caption_chart = "Relative performance: HL vs SLV (last ~2 months, close rebased to 100)."
    with open(CHART_FILENAME, "rb") as f:
        await bot.send_photo(chat_id=TELEGRAM_CHAT_ID, photo=f, caption=caption_chart)


# Jupyter-safe runner
loop = asyncio.get_event_loop()
if loop.is_running():
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())
